In [9]:
import json
from datetime import datetime
from app.utils.vercel_kv import KV
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain_openai import ChatOpenAI
from app.models import User, Message

kv = KV()
model = ChatOpenAI(model_name="gpt-4o")

In [10]:
system_prompts = [
    "You are an Fitness Coach that helps the user loose weight. This is your first conversation with the user. Keep your answers short and simple and try to find out if the user is already tracking their steps. After that, try to learn about the first name, age, gender and height of the user. ",
]

In [25]:
kv.set("+41799506553", None)
kv.get("+41799506553")

'null'

In [29]:
def _message_from_dict(message: dict) -> BaseMessage:
    _type = message["type"]
    if _type == "human":
        return HumanMessage(**message)
    elif _type == "ai":
        return AIMessage(**message)
    elif _type == "system":
        return SystemMessage(**message)
    # Add other message types as needed
    else:
        raise ValueError(f"Got unexpected message type: {_type}")

def get_user(wa_id):
    user_object = kv.get(wa_id)
    
    if user_object is None or user_object == "" or user_object == "null":
        user = User()
    else:
        user_dict = json.loads(user_object)
        if 'messages' in user_dict:
            user_dict['messages'] = [
                {
                    **message,
                    'base_message': _message_from_dict(message['base_message'])
                }
                for message in user_dict['messages']
            ]
        user = User(**user_dict)

    return user

def update_user(wa_id, user, user_message, reply_message):
    user.messages.append(user_message)
    user.messages.append(reply_message)
    kv.set(wa_id, user.dict())

def get_system_message(user):
    return SystemMessage(system_prompts[0])

def process_message(message, wa_id):
    user = get_user(wa_id)
    sys = Message(base_message=get_system_message(user))
    hum = Message(time=datetime.now(), base_message=HumanMessage(message))
    
    messages = user.messages[-5:] + [sys, hum]
    
    # for message in messages:
    #     print(message.base_message.type + ": " + message.base_message.content)
    
    
    base_messages = [message.base_message for message in messages]
    reply = Message(time=datetime.now(), base_message=model.invoke(base_messages))
    
    update_user(wa_id, user, hum, reply)
    
    print("User: ", message)
    print("FitChat: ", reply.base_message.content)
    
process_message("Hiii", "+41799506553")
process_message("Yes, im tracking my steps", "+41799506553")
process_message("Yves", "+41799506553")

User:  Hiii
FitChat:  Hi there! Are you currently tracking your steps?
User:  Yes, im tracking my steps
FitChat:  That's awesome! What's your first name?
User:  Yves
FitChat:  Nice to meet you, Yves! You mentioned you're tracking your steps. How many steps are you aiming for each day?


In [5]:
test = kv.get("+41799506553")
print(test)

{"paid": false, "user_data": {"first_name": null, "age": null, "gender": null, "height": null}, "activity_log": null, "messages": [{"time": "2024-07-05T17:43:32.336889", "base_message": {"content": "You are an Fitness Coach that helps the user loose weight. This is your first conversation with the user. Keep your answers short and simple and try to find out if the user is already tracking their steps. After that, try to learn about the first name, age, gender and height of the user. ", "additional_kwargs": {}, "response_metadata": {}, "type": "system", "name": null, "id": null}}, {"time": "2024-07-05T17:43:32.336911", "base_message": {"content": "Hiii", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}, {"time": "2024-07-05T17:43:32.336943", "base_message": {"content": "Hello! Welcome to your fitness journey. Are you currently tracking your daily steps?", "additional_kwargs": {}, "response_metadata": {"token_usage": {"comple